In [1]:
using Random
using Distributions
using LinearAlgebra
using DataFrames
using StatsBase
using PDMats
using CSV
using Dates
using JuMP
using Ipopt
import SpecialFunctions: loggamma
import MathOptInterface as MOI

In [2]:
# Data-generating parameters
pi_true = [
  vec([0.5731372264896774 0.3584768992248216 0.01085525821402486 0.05753061607147426]),
  vec([0.0370388230815912 0.889042514556095 0.002007726412896501 0.07191093594941617]),
  vec([0.006318513965246314 0.0071812424323565 0.7291813565982903 0.257318887004104]),
  vec([0.01157929936688799 0.08442420898922519 0.1134931812666756 0.7905033103772089])
]

mu_true = [
  vec([0.006734506602107449 0.003381498351761594 0.01251958486816306 0.007574169914426993]),
  vec([0.0009441833395542966 0.0003530895525309008 -0.001122619477144013 3.041564764696133e-06]),
  vec([-0.0001357129572322458 0.0001789316615930574 -0.0002532572436555386 -0.0005170677905046179]),
  vec([0.001248987177806565 0.001216024923786339 0.0009936349596729377 -0.0002090026527542731])
]

Sigma_true = [
  [
      0.002054506019709262 0.0005749811760326436 0.0005816549429042377 0.0006146245962203936;
      0.0005749811760326436 0.002379851613164497 0.001639530029524073 0.003591993629918802;
      0.0005816549429042377 0.001639530029524073 0.003565602256146811 0.002759543733341633;
      0.0006146245962203936 0.003591993629918802 0.002759543733341633 0.01157320665447896
],
  [
      0.0002717377294919257 5.902118513846123e-05 6.848107986651424e-05 2.509832559253089e-05;
      5.902118513846123e-05 0.0002444336297386139 0.0001685020755143676 0.0001883782584247908;
      6.848107986651424e-05 0.0001685020755143676 0.0005004297863327951 0.0002691805211144379;
      2.509832559253089e-05 0.0001883782584247908 0.0002691805211144379 0.0008924838375103492
],
  [
      0.000402320019330142 0.0001061006439395391 6.300527335193929e-05 6.423870914075368e-05;
      0.0001061006439395391 0.0003125691916858654 0.000169295996800317 0.0001073321464084567;
      6.300527335193929e-05 0.000169295996800317 0.0002752254194025297 8.940431341503119e-05;
      6.423870914075368e-05 0.0001073321464084567 8.940431341503119e-05 0.0001430435998786714
],
  [
      6.386967467667764e-05 1.065760662021473e-05 1.730953121602137e-05 -2.450550198361974e-06;
      1.065760662021473e-05 6.364941106262232e-05 4.037048036193801e-05 1.303416250098312e-05;
      1.730953121602137e-05 4.037048036193801e-05 0.0001441498727498638 4.898473242735126e-05;
      -2.450550198361974e-06 1.303416250098312e-05 4.898473242735126e-05 0.0002539259627920557
]
]

4-element Vector{Matrix{Float64}}:
 [0.002054506019709262 0.0005749811760326436 0.0005816549429042377 0.0006146245962203936; 0.0005749811760326436 0.002379851613164497 0.001639530029524073 0.003591993629918802; 0.0005816549429042377 0.001639530029524073 0.003565602256146811 0.002759543733341633; 0.0006146245962203936 0.003591993629918802 0.002759543733341633 0.01157320665447896]
 [0.0002717377294919257 5.902118513846123e-5 6.848107986651424e-5 2.509832559253089e-5; 5.902118513846123e-5 0.0002444336297386139 0.0001685020755143676 0.0001883782584247908; 6.848107986651424e-5 0.0001685020755143676 0.0005004297863327951 0.0002691805211144379; 2.509832559253089e-5 0.0001883782584247908 0.0002691805211144379 0.0008924838375103492]
 [0.000402320019330142 0.0001061006439395391 6.30052733519393e-5 6.423870914075368e-5; 0.0001061006439395391 0.0003125691916858654 0.000169295996800317 0.0001073321464084567; 6.30052733519393e-5 0.000169295996800317 0.0002752254194025297 8.940431341503119e-5; 6.4238

In [3]:
# Simulate one dataset
function simulate_series(pi_true, mu_true, Sigma_true; Total_T=1764)
    K = length(pi_true)
    Dloc = length(mu_true[1])
    S = Vector{Int}(undef, Total_T)
    Yf = Matrix{Float64}(undef, Total_T, Dloc)

    S[1] = rand(1:K)
    Yf[1, :] .= rand(MvNormal(vec(mu_true[S[1]]), Sigma_true[S[1]]))
    for t in 2:Total_T
        S[t] = rand(Categorical(vec(pi_true[S[t-1]])))
        Yf[t, :] .= rand(MvNormal(vec(mu_true[S[t]]), Sigma_true[S[t]]))
    end
    return S, Yf
end

function backtest_once(seed::Int; Total_T=1764, lookback=504, H=20, prefix="EW")
    Random.seed!(seed)

    # Simulate data for this seed
    _, Y_full = simulate_series(pi_true, mu_true, Sigma_true; Total_T=Total_T)
    Y      = copy(Y_full[1:lookback, :])
    Y_test = copy(Y_full[lookback+1:end, :])

    # Dimensions
    D   = size(Y, 2)
    T   = lookback
    n_days = size(Y_test, 1)

    # Starting portfolio state
    x = fill(1/D, D)
    h = fill(1000/D, D)
    V = [sum(h)]
    days_traded = 0

    # Transaction costs
    kappa = fill(0.001, D)

    # Per-test CSV
    cd(joinpath(@__DIR__, "EW"))
    test_csv = "$(prefix) $(seed).csv"
    open(test_csv, "w") do io
        hdr = ["iter", "fees", "V_new"]
        println(io, join(hdr, ","))
    end

    for i in 1:n_days
        # Rebalance to equal weights
        x_target    = fill(1/D, D)
        V_pre       = sum(h)
        h_proposal  = x_target * V_pre
        fees        = sum(kappa .* abs.(h_proposal .- h))
        V_post      = V_pre - fees
    
        h .= x_target .* V_post
        x = h ./ sum(h)
    
        # Apply next‐day returns
        r_next = view(Y_test, 1, :)
        h      .= h      .* (1 .+ r_next)
        V_new = sum(h)
        x = h ./ V_new
    
        # Roll returns forward
        Y      = vcat(Y[2:end, :], transpose(r_next))
        Y_test = Y_test[2:end, :]
    
        # Record & print
        days_traded += 1
        push!(V, V_new)

        row = vcat([days_traded, fees, V_new])
        open(test_csv, "a") do io
            println(io, join(row, ","))
        end
    end

    cd(@__DIR__)

    return V[end]
end

summary_csv = "EW summary.csv"
open(summary_csv, "w") do io
    println(io, "test,final_V")
end

# Run 25 independent tests
for test in 1:25
    final_V = backtest_once(test; Total_T=1764, lookback=504, H=20, prefix="EW")
    open(summary_csv, "a") do io
        println(io, "$(test),$(final_V)")
    end
end
